# Data Cleaning the Traffic Datasets

I obtained the data from the 2015 Traffic Fatalities provided by NHTSA available __[here](https://www.kaggle.com/nhtsa/2015-traffic-fatalities)__.  This consists of 17 related csv files which contain in common unique case identifiers.  The accompanying NHTSA documentation defines the meanings of the codes and abbreviations used throughout each dataset.  

Note:  Relevant portions from this report are interspersed throughout the code below for context.

Although I did not use all 17 datasets, most of them were used and loaded using pd.read_csv(). The goal was to use data that could demonstrate what factors were most important in fatal accidents, i.e., environmental factors versus human elements, as well as to choose those factors that might be useful in predicting whether a “serious” fatality would occur (greater than 1 fatality in a given accident).  

I began by merging the datasets chosen using the merge function in pandas. The datasets used were Accident, Vehicle, Person, Distract, Drimpair, Factor, Maneuver, Violatn and Vision.  The first two are the principal datasets, containing the primary references to the keys ST_CASE and VEH_NO, and were merged first.  First Vehicle was merged to Accident on ST_CASE into a new dataframe called __traffic__.  Person was then merged to traffic on ST_CASE and VEH_NO. The remaining datasets were placed into a list (to_merge = [distract, dr_impair, factor, maneuver, violation, vision]) and then merged to __traffic__ via a for loop on ST_CASE and VEH_NO.  All merges were left merges, except for the Person merge, which was an inner merge necessary to get rid of NA values after merge resulting from no values for certain rare person types such as pedestrians.  (When doing machine learning, the person type will likely be restricted to only drivers, but all data is being kept for illustrative/statistical analysis purposes for now.)  

After the merge I checked for missing data using traffic.info(), and no data was missing (these were fairly clean datasets to begin with). I then checked for outliers using traffic.describe().  The only apparent large differences between mean and median were in the last six columns, likely due to the number of unknowns being represented by 99. 
Therefore numerical outliers appear to only be an issue for columns with numerical, rather than categorical data (usually mixed with codes for “Unknown”-type values.)  The majority of the columns are categorical, but a few are numerical. I checked for outliers in the numerical columns using value_counts (traffic.loc[:, ['TRAV_SP', 'FATALS', 'DRUNK_DR', 'VSPD_LIM', 'DEATHS', 'AGE']].apply(pd.Series.value_counts)).  TRAV_SP (the travel speed), VSPD_LIM (speed limit) and AGE all contained inordinately large values that were actually codes for Unknown-type values (999, 998, etc).  These were converted to zero in the first two. In AGE, zero indicated infants, so the zeroes here were first converted to 1 (one-year old babies), so as not to confuse these with the unknowns to be converted to zero.  

Finally, in the interest of performing machine learning later, I created a number of columns that were binary equivalents of existing columns for simplicity (for example, whether or not there was a visual obstruction, rather than displaying all possible types of visual obstruction).  All binary equivalent columns were prefaced with the “BIN\_” prefix.  However, the original columns were also kept for illustrative/statistical analysis purposes where the greater level of detail may be preferred.   

The final column created was SERIOUS_FATALS.  This column was based on the FATALS column where FATALS (number of fatalities in a given accident) was greater than 1.  This variable is the one that I intend to use as the variable to be predicted when applying machine learning.  


### Report Excerpts

I obtained the data from the 2015 Traffic Fatalities provided by NHTSA available __[here](https://www.kaggle.com/nhtsa/2015-traffic-fatalities)__.  This consists of 17 related csv files which contain in common unique case identifiers.  The accompanying NHTSA documentation defines the meanings of the codes and abbreviations used throughout each dataset.  

Although I did not use all 17 datasets, most of them were used and loaded using pd.read_csv().  The goal was to use data that could demonstrate what factors were most important in fatal accidents, i.e., environmental factors versus human elements, as well as to choose those factors that might be useful in predicting whether a “serious” fatality would occur (greater than 1 fatality in a given accident).  


In [1]:
import numpy as np
import pandas as pd

#Import accident dataset
accident = pd.read_csv("accident.csv")
accident.head()

,STATE,ST_CASE,VE_TOTAL,VE_FORMS,PVH_INVL,PEDS,PERNOTMVIT,PERMVIT,PERSONS,COUNTY,...,NOT_MIN,ARR_HOUR,ARR_MIN,HOSP_HR,HOSP_MN,CF1,CF2,CF3,FATALS,DRUNK_DR
0,1,10001,1,1,0,0,0,1,1,127,...,99,2,58,88,88,0,0,0,1,1
1,1,10002,1,1,0,0,0,1,1,83,...,99,22,20,88,88,0,0,0,1,0
2,1,10003,1,1,0,0,0,2,2,11,...,99,1,45,99,99,0,0,0,1,1
3,1,10004,1,1,0,0,0,1,1,45,...,99,1,15,88,88,0,0,0,1,1
4,1,10005,2,2,0,0,0,2,2,45,...,99,7,16,88,88,0,0,0,1,0


In [2]:
#Choosing appropriate columns for accident
accident = accident[['ST_CASE', 'MONTH', 'YEAR', 'DAY_WEEK', 'HOUR', 'RUR_URB', 'ROUTE', 'RELJCT1',
       'RELJCT2', 'WRK_ZONE', 'LGT_COND', 'WEATHER1', 'WEATHER2', 'WEATHER',
       'CF1', 'CF2', 'CF3', 'FATALS', 'DRUNK_DR']]

#Checking for null values/incomplete columns
accident.info()
accident.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32166 entries, 0 to 32165
Data columns (total 19 columns):
ST_CASE     32166 non-null int64
MONTH       32166 non-null int64
YEAR        32166 non-null int64
DAY_WEEK    32166 non-null int64
HOUR        32166 non-null int64
RUR_URB     32166 non-null int64
ROUTE       32166 non-null int64
RELJCT1     32166 non-null int64
RELJCT2     32166 non-null int64
WRK_ZONE    32166 non-null int64
LGT_COND    32166 non-null int64
WEATHER1    32166 non-null int64
WEATHER2    32166 non-null int64
WEATHER     32166 non-null int64
CF1         32166 non-null int64
CF2         32166 non-null int64
CF3         32166 non-null int64
FATALS      32166 non-null int64
DRUNK_DR    32166 non-null int64
dtypes: int64(19)
memory usage: 4.7 MB


,ST_CASE,MONTH,YEAR,DAY_WEEK,HOUR,RUR_URB,ROUTE,RELJCT1,RELJCT2,WRK_ZONE,LGT_COND,WEATHER1,WEATHER2,WEATHER,CF1,CF2,CF3,FATALS,DRUNK_DR
0,10001,1,2015,5,2,1,3,0,1,0,2,1,0,1,0,0,0,1,1
1,10002,1,2015,5,22,1,1,0,1,0,2,10,0,10,0,0,0,1,0
2,10003,1,2015,5,1,1,2,0,1,0,2,1,0,1,0,0,0,1,1
3,10004,1,2015,1,0,1,3,0,1,0,2,10,0,10,0,0,0,1,1
4,10005,1,2015,4,7,2,2,0,2,0,1,1,0,1,0,0,0,1,0


In [3]:
#Import vehicle dataframe and choosing appropriate columns
vehicle = pd.read_csv("vehicle.csv", encoding = "ISO-8859-1")
vehicle = vehicle[['ST_CASE', 'VEH_NO', 'HAZ_INV', 'TRAV_SP',
       'SPEEDREL', 'VSPD_LIM', 'VPROFILE', 'VPAVETYP', 'VSURCOND', 'VTRAFCON',
       'VTCONT_F', 'DEATHS']]

vehicle.info()
vehicle.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48923 entries, 0 to 48922
Data columns (total 12 columns):
ST_CASE     48923 non-null int64
VEH_NO      48923 non-null int64
HAZ_INV     48923 non-null int64
TRAV_SP     48923 non-null int64
SPEEDREL    48923 non-null int64
VSPD_LIM    48923 non-null int64
VPROFILE    48923 non-null int64
VPAVETYP    48923 non-null int64
VSURCOND    48923 non-null int64
VTRAFCON    48923 non-null int64
VTCONT_F    48923 non-null int64
DEATHS      48923 non-null int64
dtypes: int64(12)
memory usage: 4.5 MB


,ST_CASE,VEH_NO,HAZ_INV,TRAV_SP,SPEEDREL,VSPD_LIM,VPROFILE,VPAVETYP,VSURCOND,VTRAFCON,VTCONT_F,DEATHS
0,10001,1,1,55,0,55,1,2,1,0,0,1
1,10002,1,1,70,4,70,1,2,2,0,0,1
2,10003,1,1,80,0,55,1,2,1,0,0,1
3,10004,1,1,75,0,55,1,2,1,0,0,1
4,10005,1,1,15,0,65,6,2,1,0,0,1


In [4]:
#Import person dataset and choose appropriate columns
person = pd.read_csv("person.csv")
person = person[['ST_CASE', 'VEH_NO', 'PER_NO', 'AGE', 'PER_TYP', 'INJ_SEV', 'REST_USE', 
                 'REST_MIS','DRUGS', 'P_SF1', 'P_SF2', 'P_SF3']]
person.info()
person.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80587 entries, 0 to 80586
Data columns (total 12 columns):
ST_CASE     80587 non-null int64
VEH_NO      80587 non-null int64
PER_NO      80587 non-null int64
AGE         80587 non-null int64
PER_TYP     80587 non-null int64
INJ_SEV     80587 non-null int64
REST_USE    80587 non-null int64
REST_MIS    80587 non-null int64
DRUGS       80587 non-null int64
P_SF1       80587 non-null int64
P_SF2       80587 non-null int64
P_SF3       80587 non-null int64
dtypes: int64(12)
memory usage: 7.4 MB


,ST_CASE,VEH_NO,PER_NO,AGE,PER_TYP,INJ_SEV,REST_USE,REST_MIS,DRUGS,P_SF1,P_SF2,P_SF3
0,10001,1,1,68,1,4,7,0,9,0,0,0
1,10002,1,1,49,1,4,7,0,0,0,0,0
2,10003,1,1,31,1,4,7,0,9,0,0,0
3,10003,1,2,20,2,2,7,0,8,0,0,0
4,10004,1,1,40,1,4,7,0,9,0,0,0


I began by merging the datasets chosen using the merge function in pandas. The datasets used were Accident, Vehicle, Person, Distract, Drimpair, Factor, Maneuver, Violatn and Vision.  The first two are the principal datasets, containing the primary references to the keys ST_CASE and VEH_NO, and were merged first.  First Vehicle was merged to Accident on ST_CASE into a new dataframe called __traffic__.  

In [5]:
#First merge:  accident and vehicle on ST_CASE
traffic = accident.merge(vehicle, on="ST_CASE", how="left")
traffic.info()
traffic.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48923 entries, 0 to 48922
Data columns (total 30 columns):
ST_CASE     48923 non-null int64
MONTH       48923 non-null int64
YEAR        48923 non-null int64
DAY_WEEK    48923 non-null int64
HOUR        48923 non-null int64
RUR_URB     48923 non-null int64
ROUTE       48923 non-null int64
RELJCT1     48923 non-null int64
RELJCT2     48923 non-null int64
WRK_ZONE    48923 non-null int64
LGT_COND    48923 non-null int64
WEATHER1    48923 non-null int64
WEATHER2    48923 non-null int64
WEATHER     48923 non-null int64
CF1         48923 non-null int64
CF2         48923 non-null int64
CF3         48923 non-null int64
FATALS      48923 non-null int64
DRUNK_DR    48923 non-null int64
VEH_NO      48923 non-null int64
HAZ_INV     48923 non-null int64
TRAV_SP     48923 non-null int64
SPEEDREL    48923 non-null int64
VSPD_LIM    48923 non-null int64
VPROFILE    48923 non-null int64
VPAVETYP    48923 non-null int64
VSURCOND    48923 non-null int64


,ST_CASE,MONTH,YEAR,DAY_WEEK,HOUR,RUR_URB,ROUTE,RELJCT1,RELJCT2,WRK_ZONE,...,HAZ_INV,TRAV_SP,SPEEDREL,VSPD_LIM,VPROFILE,VPAVETYP,VSURCOND,VTRAFCON,VTCONT_F,DEATHS
0,10001,1,2015,5,2,1,3,0,1,0,...,1,55,0,55,1,2,1,0,0,1
1,10002,1,2015,5,22,1,1,0,1,0,...,1,70,4,70,1,2,2,0,0,1
2,10003,1,2015,5,1,1,2,0,1,0,...,1,80,0,55,1,2,1,0,0,1
3,10004,1,2015,1,0,1,3,0,1,0,...,1,75,0,55,1,2,1,0,0,1
4,10005,1,2015,4,7,2,2,0,2,0,...,1,15,0,65,6,2,1,0,0,1


Person was then merged to traffic on ST_CASE and VEH_NO. 

In [6]:
#Second merge: traffic with person on ST_CASE and VEH_NO
traffic = traffic.merge(person, on=["ST_CASE", "VEH_NO"], how="inner")
traffic.info()
traffic.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 73390 entries, 0 to 73389
Data columns (total 40 columns):
ST_CASE     73390 non-null int64
MONTH       73390 non-null int64
YEAR        73390 non-null int64
DAY_WEEK    73390 non-null int64
HOUR        73390 non-null int64
RUR_URB     73390 non-null int64
ROUTE       73390 non-null int64
RELJCT1     73390 non-null int64
RELJCT2     73390 non-null int64
WRK_ZONE    73390 non-null int64
LGT_COND    73390 non-null int64
WEATHER1    73390 non-null int64
WEATHER2    73390 non-null int64
WEATHER     73390 non-null int64
CF1         73390 non-null int64
CF2         73390 non-null int64
CF3         73390 non-null int64
FATALS      73390 non-null int64
DRUNK_DR    73390 non-null int64
VEH_NO      73390 non-null int64
HAZ_INV     73390 non-null int64
TRAV_SP     73390 non-null int64
SPEEDREL    73390 non-null int64
VSPD_LIM    73390 non-null int64
VPROFILE    73390 non-null int64
VPAVETYP    73390 non-null int64
VSURCOND    73390 non-null int64


,ST_CASE,MONTH,YEAR,DAY_WEEK,HOUR,RUR_URB,ROUTE,RELJCT1,RELJCT2,WRK_ZONE,...,PER_NO,AGE,PER_TYP,INJ_SEV,REST_USE,REST_MIS,DRUGS,P_SF1,P_SF2,P_SF3
0,10001,1,2015,5,2,1,3,0,1,0,...,1,68,1,4,7,0,9,0,0,0
1,10002,1,2015,5,22,1,1,0,1,0,...,1,49,1,4,7,0,0,0,0,0
2,10003,1,2015,5,1,1,2,0,1,0,...,1,31,1,4,7,0,9,0,0,0
3,10003,1,2015,5,1,1,2,0,1,0,...,2,20,2,2,7,0,8,0,0,0
4,10004,1,2015,1,0,1,3,0,1,0,...,1,40,1,4,7,0,9,0,0,0


The remaining datasets were placed into a list (to_merge = [distract, dr_impair, factor, maneuver, violation, vision]) and then merged to __traffic__ via a for loop on ST_CASE and VEH_NO.  All merges were left merges, except for the Person merge, which was an inner merge necessary to get rid of NA values after merge resulting from no values for certain rare person types such as pedestrians.  (When doing machine learning, the person type will likely be restricted to only drivers, but all data is being kept for illustrative/statistical analysis purposes for now.)  

After the merge I checked for missing data using traffic.info(), and no data was missing (these were fairly clean datasets to begin with).  

In [7]:
#Load remaining datasets and prepare list for merge
distract = pd.read_csv("distract.csv", usecols=[1, 2, 3])
dr_impair = pd.read_csv("drimpair.csv", usecols=[1, 2, 3])
factor = pd.read_csv("factor.csv", usecols=[1, 2, 3])
maneuver = pd.read_csv("maneuver.csv", usecols=[1, 2, 3])
violation = pd.read_csv("violatn.csv", usecols=[1, 2, 3])
vision = pd.read_csv("vision.csv", usecols=[1, 2, 3])

to_merge = [distract, dr_impair, factor, maneuver, violation, vision]

In [8]:
#Merge remaining datasets

for i in to_merge:
    traffic = traffic.merge(i, on=["ST_CASE", "VEH_NO"], how="left")
    
traffic.info()
traffic.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79619 entries, 0 to 79618
Data columns (total 46 columns):
ST_CASE     79619 non-null int64
MONTH       79619 non-null int64
YEAR        79619 non-null int64
DAY_WEEK    79619 non-null int64
HOUR        79619 non-null int64
RUR_URB     79619 non-null int64
ROUTE       79619 non-null int64
RELJCT1     79619 non-null int64
RELJCT2     79619 non-null int64
WRK_ZONE    79619 non-null int64
LGT_COND    79619 non-null int64
WEATHER1    79619 non-null int64
WEATHER2    79619 non-null int64
WEATHER     79619 non-null int64
CF1         79619 non-null int64
CF2         79619 non-null int64
CF3         79619 non-null int64
FATALS      79619 non-null int64
DRUNK_DR    79619 non-null int64
VEH_NO      79619 non-null int64
HAZ_INV     79619 non-null int64
TRAV_SP     79619 non-null int64
SPEEDREL    79619 non-null int64
VSPD_LIM    79619 non-null int64
VPROFILE    79619 non-null int64
VPAVETYP    79619 non-null int64
VSURCOND    79619 non-null int64


,ST_CASE,MONTH,YEAR,DAY_WEEK,HOUR,RUR_URB,ROUTE,RELJCT1,RELJCT2,WRK_ZONE,...,DRUGS,P_SF1,P_SF2,P_SF3,MDRDSTRD,DRIMPAIR,MFACTOR,MDRMANAV,MVIOLATN,MVISOBSC
0,10001,1,2015,5,2,1,3,0,1,0,...,9,0,0,0,99,0,0,98,0,0
1,10002,1,2015,5,22,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,10003,1,2015,5,1,1,2,0,1,0,...,9,0,0,0,0,9,0,0,0,0
3,10003,1,2015,5,1,1,2,0,1,0,...,8,0,0,0,0,9,0,0,0,0
4,10004,1,2015,1,0,1,3,0,1,0,...,9,0,0,0,99,9,0,0,0,0


In [9]:
#View all columns
traffic.columns


Index(['ST_CASE', 'MONTH', 'YEAR', 'DAY_WEEK', 'HOUR', 'RUR_URB', 'ROUTE',
       'RELJCT1', 'RELJCT2', 'WRK_ZONE', 'LGT_COND', 'WEATHER1', 'WEATHER2',
       'WEATHER', 'CF1', 'CF2', 'CF3', 'FATALS', 'DRUNK_DR', 'VEH_NO',
       'HAZ_INV', 'TRAV_SP', 'SPEEDREL', 'VSPD_LIM', 'VPROFILE', 'VPAVETYP',
       'VSURCOND', 'VTRAFCON', 'VTCONT_F', 'DEATHS', 'PER_NO', 'AGE',
       'PER_TYP', 'INJ_SEV', 'REST_USE', 'REST_MIS', 'DRUGS', 'P_SF1', 'P_SF2',
       'P_SF3', 'MDRDSTRD', 'DRIMPAIR', 'MFACTOR', 'MDRMANAV', 'MVIOLATN',
       'MVISOBSC'],
      dtype='object')

I then checked for outliers using traffic.describe().  The only apparent large differences between mean and median were in the last six columns, likely due to the number of unknowns being represented by 99. 
Therefore numerical outliers appear to only be an issue for columns with numerical, rather than categorical data (usually mixed with codes for “Unknown”-type values.)  The majority of the columns are categorical, but a few are numerical. 

In [10]:
#Check for outliers--only apparent large differences between mean and median are in last 6 columns.
#Likely due to number of unknowns being represented by 99. 
#Therefore numerical outliers only an issue for columns with numerical data (usually mixed with codes)
traffic.describe()

,ST_CASE,MONTH,YEAR,DAY_WEEK,HOUR,RUR_URB,ROUTE,RELJCT1,RELJCT2,WRK_ZONE,...,DRUGS,P_SF1,P_SF2,P_SF3,MDRDSTRD,DRIMPAIR,MFACTOR,MDRMANAV,MVIOLATN,MVISOBSC
count,79619.000000,79619.000000,79619.0,79619.000000,79619.000000,79619.000000,79619.000000,79619.000000,79619.000000,79619.000000,...,79619.000000,79619.000000,79619.000000,79619.000000,79619.000000,79619.000000,79619.000000,79619.000000,79619.000000,79619.000000
mean,278837.884801,6.773911,2015.0,4.124882,13.410379,1.936523,3.594506,0.051746,2.256283,0.039086,...,4.599543,0.190846,0.065964,0.065901,26.363041,23.386817,6.010123,46.106319,7.012836,3.987239
std,162125.918213,3.334996,0.0,2.115920,8.730511,1.716789,2.056005,0.258368,4.590703,0.305574,...,4.024692,3.819493,2.553484,2.553424,42.805420,40.416534,23.422545,48.537010,21.289601,19.250018
min,10001.000000,1.000000,2015.0,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,122391.000000,4.000000,2015.0,2.000000,8.000000,1.000000,2.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,280220.000000,7.000000,2015.0,4.000000,14.000000,2.000000,3.000000,0.000000,1.000000,0.000000,...,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000
75%,420715.000000,10.000000,2015.0,6.000000,19.000000,2.000000,5.000000,0.000000,2.000000,0.000000,...,8.000000,0.000000,0.000000,0.000000,93.000000,9.000000,0.000000,98.000000,0.000000,0.000000
max,560130.000000,12.000000,2015.0,7.000000,99.000000,9.000000,9.000000,9.000000,99.000000,4.000000,...,9.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000


I checked for outliers in the numerical columns using value_counts (traffic.loc[:, ['TRAV_SP', 'FATALS', 'DRUNK_DR', 'VSPD_LIM', 'DEATHS', 'AGE']].apply(pd.Series.value_counts)).  TRAV_SP (the travel speed), VSPD_LIM (speed limit) and AGE all contained inordinately large values that were actually codes for Unknown-type values (999, 998, etc).  These were converted to zero in the first two.

In [11]:
#Check numerical columns for outliers
traffic.loc[:, ['TRAV_SP', 'FATALS', 'DRUNK_DR', 'VSPD_LIM', 'DEATHS', 'AGE']].apply(pd.Series.value_counts)

,TRAV_SP,FATALS,DRUNK_DR,VSPD_LIM,DEATHS,AGE
0,3156,NaN,57781.0,566.0,35151.0,358.0
1,36,69811.0,20878.0,NaN,38654.0,458.0
2,36,7379.0,950.0,NaN,4388.0,412.0
3,41,1539.0,10.0,NaN,992.0,441.0
4,13,478.0,NaN,NaN,251.0,393.0
5,735,264.0,NaN,24.0,114.0,400.0
6,25,115.0,NaN,NaN,36.0,403.0
7,32,NaN,NaN,NaN,NaN,371.0
8,41,18.0,NaN,NaN,18.0,383.0
9,12,NaN,NaN,NaN,NaN,427.0


In [12]:
#Change unknowns in TRAV_SP from numerical codes to 0 (codes and actual numerical data mixed here)
traffic.loc[((traffic['TRAV_SP'] == 998) | (traffic['TRAV_SP'] == 999)), 'TRAV_SP'] = 0

#Confirm changes
traffic.loc[((traffic['TRAV_SP'] == 998) | (traffic['TRAV_SP'] == 999)), 'TRAV_SP']


Series([], Name: TRAV_SP, dtype: int64)

In [13]:
#Change > 151 in TRAV_SP from numerical code 997 to 152 (codes and actual numerical data mixed here)
traffic.loc[(traffic['TRAV_SP'] == 997), 'TRAV_SP'] = 152

#Confirm changes
traffic.loc[(traffic['TRAV_SP'] == 997), 'TRAV_SP']



Series([], Name: TRAV_SP, dtype: int64)

In [14]:
#Change unknowns in VSPD_LIM from numerical codes to 0 (codes and actual numerical data mixed here)
traffic.loc[((traffic['VSPD_LIM'] == 98) | (traffic['VSPD_LIM'] == 99)), 'VSPD_LIM'] = 0

#Confirm changes
traffic.loc[((traffic['VSPD_LIM'] == 98) | (traffic['VSPD_LIM'] == 99)), 'VSPD_LIM']

Series([], Name: VSPD_LIM, dtype: int64)

In AGE, zero indicated infants, so the zeroes here were first converted to 1 (one-year old babies), so as not to confuse these with the unknowns to be converted to zero.  

In [15]:
#Change zeros in AGE (indicating infants)to 1 to not confuse with later conversion of unknowns to 0
traffic.loc[(traffic['AGE'] == 0), 'AGE'] = 1

#Confirm changes
traffic.loc[(traffic['AGE'] == 0), 'AGE']


Series([], Name: AGE, dtype: int64)

In [16]:
#Change unknowns in AGE from numerical codes to 0 (codes and actual numerical data mixed here)
traffic.loc[((traffic['AGE'] == 998) | (traffic['AGE'] == 999)), 'AGE'] = 0

#Confirm changes
traffic.loc[((traffic['AGE'] == 998) | (traffic['AGE'] == 999)), 'AGE']


Series([], Name: AGE, dtype: int64)

In [17]:
#Confirm corrections in numerical columns 
traffic.loc[:, ['TRAV_SP', 'FATALS', 'DRUNK_DR', 'VSPD_LIM', 'DEATHS', 'AGE']].apply(pd.Series.value_counts)

,TRAV_SP,FATALS,DRUNK_DR,VSPD_LIM,DEATHS,AGE
0,51150,NaN,57781.0,3864.0,35151.0,1619.0
1,36,69811.0,20878.0,NaN,38654.0,816.0
2,36,7379.0,950.0,NaN,4388.0,412.0
3,41,1539.0,10.0,NaN,992.0,441.0
4,13,478.0,NaN,NaN,251.0,393.0
5,735,264.0,NaN,24.0,114.0,400.0
6,25,115.0,NaN,NaN,36.0,403.0
7,32,NaN,NaN,NaN,NaN,371.0
8,41,18.0,NaN,NaN,18.0,383.0
9,12,NaN,NaN,NaN,NaN,427.0


Finally, in the interest of performing machine learning later, I created a number of columns that were binary equivalents of existing columns for simplicity (for example, whether or not there was a visual obstruction, rather than displaying all possible types of visual obstruction).  All binary equivalent columns were prefaced with the “BIN\_” prefix.  However, the original columns were also kept for illustrative/statistical analysis purposes where the greater level of detail may be preferred.   

In [18]:
#Convert WRK_ZONE to binary (construction zone or not)
#Copy column to new column BIN_WRK_ZONE
traffic['BIN_WRK_ZONE'] = traffic.loc[:, 'WRK_ZONE']

#Convert other substantive values to 1 (Yes)
traffic.loc[(traffic['BIN_WRK_ZONE'] > 0), 'BIN_WRK_ZONE'] = 1

#Confirm changes made
traffic.loc[(traffic['BIN_WRK_ZONE'] > 1), 'BIN_WRK_ZONE']


Series([], Name: BIN_WRK_ZONE, dtype: int64)

In [19]:
#Convert SPEEDREL to binary (speed-related or not)
#Copy column to new column BIN_SPEEDREL 
traffic['BIN_SPEEDREL'] = traffic.loc[:, 'SPEEDREL']
#Convert unknown values to 0 (No)
traffic.loc[((traffic['BIN_SPEEDREL'] == 8) | (traffic['BIN_SPEEDREL'] == 9)), 'BIN_SPEEDREL'] = 0

#Confirm changes made
traffic.loc[((traffic['BIN_SPEEDREL'] == 8) | (traffic['BIN_SPEEDREL'] == 9)), 'BIN_SPEEDREL']


Series([], Name: BIN_SPEEDREL, dtype: int64)

In [20]:
#Convert BIN_SPEEDREL values greater than 1 to 1 (Yes)
traffic.loc[(traffic['BIN_SPEEDREL'] > 1), 'BIN_SPEEDREL'] = 1

#Confirm changes made
traffic.loc[(traffic['BIN_SPEEDREL'] > 1), 'BIN_SPEEDREL']


Series([], Name: BIN_SPEEDREL, dtype: int64)

In [21]:
#Convert REST_USE to binary (used restraints or didn't)
#Copy column to new column BIN_REST_USE 
traffic['BIN_REST_USE'] = traffic.loc[:, 'REST_USE']

#Convert unknown values to 0 (No).  Values 7, 13, 14, 15, 17, 98 and 99 relate to unknowns.
traffic.loc[(traffic['BIN_REST_USE'].isin([7, 13, 14, 15, 17, 98, 99])), 'BIN_REST_USE'] = 0

#Confirm changes made
traffic.loc[(traffic['BIN_REST_USE'].isin([7, 13, 14, 15, 17, 98, 99])), 'BIN_REST_USE']



Series([], Name: BIN_REST_USE, dtype: int64)

In [22]:
#Convert remaining numbers greater than 0 to 1 (Yes)
traffic.loc[(traffic['BIN_REST_USE'] > 0), 'BIN_REST_USE'] = 1

#Confirm changes made
traffic.loc[(traffic['BIN_REST_USE'] > 1), 'BIN_REST_USE']


Series([], Name: BIN_REST_USE, dtype: int64)

In [23]:
#Convert DRUGS to binary (used drugs or didn't)
#Copy column to new column BIN_DRUGS 
traffic['BIN_DRUGS'] = traffic.loc[:, 'DRUGS']

#Convert not reported/unknown values to 0 (No)
traffic.loc[(traffic['BIN_DRUGS'] == 8) | (traffic['BIN_DRUGS'] == 9), 'BIN_DRUGS'] = 0

#Confirm changes made
traffic.loc[(traffic['BIN_DRUGS'] == 8) | (traffic['BIN_DRUGS'] == 9), 'BIN_DRUGS']


Series([], Name: BIN_DRUGS, dtype: int64)

In [24]:
#Convert P_SF1 to binary (human actions contributing to accident or not)
#Copy column to new column BIN_P_SF1
traffic['BIN_P_SF1'] = traffic.loc[:, 'P_SF1']

#Convert 99 (unknown) values to 0 (None)
traffic.loc[(traffic['BIN_P_SF1'] == 99), 'BIN_P_SF1'] = 0

#Confirm changes made
traffic.loc[(traffic['BIN_P_SF1'] == 99), 'BIN_P_SF1']


Series([], Name: BIN_P_SF1, dtype: int64)

In [25]:
#Convert remaining numbers greater than 0 to 1 (Yes)
traffic.loc[(traffic['BIN_P_SF1'] > 0), 'BIN_P_SF1'] = 1

#Confirm changes made
traffic.loc[(traffic['BIN_P_SF1'] > 1), 'BIN_P_SF1']


Series([], Name: BIN_P_SF1, dtype: int64)

In [26]:
#Drop P_SF2 and P_SF3 columns; P_SF1 is controlling as only column with substantive values 
#(SF_2 has 1 entry of value, but this is immaterial once values are converted to binary)
traffic = traffic.drop(['P_SF2', 'P_SF3'], axis=1)

#Check columns to verify
traffic.columns


Index(['ST_CASE', 'MONTH', 'YEAR', 'DAY_WEEK', 'HOUR', 'RUR_URB', 'ROUTE',
       'RELJCT1', 'RELJCT2', 'WRK_ZONE', 'LGT_COND', 'WEATHER1', 'WEATHER2',
       'WEATHER', 'CF1', 'CF2', 'CF3', 'FATALS', 'DRUNK_DR', 'VEH_NO',
       'HAZ_INV', 'TRAV_SP', 'SPEEDREL', 'VSPD_LIM', 'VPROFILE', 'VPAVETYP',
       'VSURCOND', 'VTRAFCON', 'VTCONT_F', 'DEATHS', 'PER_NO', 'AGE',
       'PER_TYP', 'INJ_SEV', 'REST_USE', 'REST_MIS', 'DRUGS', 'P_SF1',
       'MDRDSTRD', 'DRIMPAIR', 'MFACTOR', 'MDRMANAV', 'MVIOLATN', 'MVISOBSC',
       'BIN_WRK_ZONE', 'BIN_SPEEDREL', 'BIN_REST_USE', 'BIN_DRUGS',
       'BIN_P_SF1'],
      dtype='object')

In [27]:
#Convert MDRDSTRD to binary (distraction contributing to accident or not)
#Copy column to new column BIN_MDRDSTRD
traffic['BIN_MDRDSTRD'] = traffic.loc[:, 'MDRDSTRD']

#Convert 99 (unknown) and 96 (not reported) values to 0 (No)
traffic.loc[(traffic['BIN_MDRDSTRD'] == 99) | (traffic['BIN_MDRDSTRD'] == 96), 'BIN_MDRDSTRD'] = 0

#Confirm changes made
traffic.loc[(traffic['BIN_MDRDSTRD'] == 99) | (traffic['BIN_MDRDSTRD'] == 96), 'BIN_MDRDSTRD']


Series([], Name: BIN_MDRDSTRD, dtype: int64)

In [28]:
#Convert remaining numbers greater than 0 to 1 (Yes)
traffic.loc[(traffic['BIN_MDRDSTRD'] > 0), 'BIN_MDRDSTRD'] = 1

#Confirm changes made
traffic.loc[(traffic['BIN_MDRDSTRD'] > 1), 'BIN_MDRDSTRD']



Series([], Name: BIN_MDRDSTRD, dtype: int64)

In [29]:
#Convert DRIMPAIR to binary (physical impairment contributing to accident or not)
#Copy column to new column BIN_DRIMPAIR
traffic['BIN_DRIMPAIR'] = traffic.loc[:, 'DRIMPAIR']

#Convert 99 (unknown) and 98 (Not Reported) values to 0 (No)
traffic.loc[(traffic['BIN_DRIMPAIR'] == 99) | (traffic['BIN_DRIMPAIR'] == 98), 'BIN_DRIMPAIR'] = 0

#Confirm changes made
traffic.loc[(traffic['BIN_DRIMPAIR'] == 99) | (traffic['BIN_DRIMPAIR'] == 98), 'BIN_DRIMPAIR']



Series([], Name: BIN_DRIMPAIR, dtype: int64)

In [30]:
#Convert remaining numbers greater than 0 to 1 (Yes)
traffic.loc[(traffic['BIN_DRIMPAIR'] > 0), 'BIN_DRIMPAIR'] = 1

#Confirm changes made
traffic.loc[(traffic['BIN_DRIMPAIR'] > 1), 'BIN_DRIMPAIR']


Series([], Name: BIN_DRIMPAIR, dtype: int64)

In [31]:
#Convert MFACTOR to binary (pre-existing vehicle issues contributing to accident or not)
#Copy column to new column BIN_MFACTOR
traffic['BIN_MFACTOR'] = traffic.loc[:, 'MFACTOR']

#Convert 99 (unknown) and 98 (Not Reported) values to 0 (No)
traffic.loc[(traffic['BIN_MFACTOR'] == 99) | (traffic['BIN_MFACTOR'] == 98), 'BIN_MFACTOR'] = 0

#Confirm changes made
traffic.loc[(traffic['BIN_MFACTOR'] == 99) | (traffic['BIN_MFACTOR'] == 98), 'BIN_MFACTOR']


Series([], Name: BIN_MFACTOR, dtype: int64)

In [32]:
#Convert remaining numbers greater than 0 to 1 (Yes)
traffic.loc[(traffic['BIN_MFACTOR'] > 0), 'BIN_MFACTOR'] = 1

#Confirm changes made
traffic.loc[(traffic['BIN_MFACTOR'] > 1), 'BIN_MFACTOR']


Series([], Name: BIN_MFACTOR, dtype: int64)

In [33]:
#Convert MDRMANAV to binary (maneuvered to avoid accident or not)
#Copy column to new column BIN_MDRMANAV
traffic['BIN_MDRMANAV'] = traffic.loc[:, 'MDRMANAV']

#Convert 99 (unknown) and 98 (Not Reported) and 95 (unknown if driver present) values to 0 (No)
traffic.loc[(traffic['BIN_MDRMANAV'] == 99) | (traffic['BIN_MDRMANAV'] == 98) | (traffic['BIN_MDRMANAV'] == 95), 'BIN_MDRMANAV'] = 0

#Confirm changes made
traffic.loc[(traffic['BIN_MDRMANAV'] == 99) | (traffic['BIN_MDRMANAV'] == 98) | (traffic['BIN_MDRMANAV'] == 95), 'BIN_MDRMANAV']



Series([], Name: BIN_MDRMANAV, dtype: int64)

In [34]:
#Convert remaining numbers greater than 0 to 1 (Yes)
traffic.loc[(traffic['BIN_MDRMANAV'] > 0), 'BIN_MDRMANAV'] = 1

#Confirm changes made
traffic.loc[(traffic['BIN_MDRMANAV'] > 1), 'BIN_MDRMANAV']

Series([], Name: BIN_MDRMANAV, dtype: int64)

In [35]:
#Convert MVISOBSC to binary (visual obstructions or not)
#Copy column to new column BIN_MVISOBSC
traffic['BIN_MVISOBSC'] = traffic.loc[:, 'MVISOBSC']

#Convert 99 (unknown) values to 0 (No) and 95 (unknown if driver present)
traffic.loc[(traffic['BIN_MVISOBSC'] == 99) | (traffic['BIN_MVISOBSC'] == 95), 'BIN_MVISOBSC'] = 0

#Confirm changes made
traffic.loc[(traffic['BIN_MVISOBSC'] == 99) | (traffic['BIN_MVISOBSC'] == 95), 'BIN_MVISOBSC']


Series([], Name: BIN_MVISOBSC, dtype: int64)

In [36]:
#Convert remaining numbers greater than 0 to 1 (Yes)
traffic.loc[(traffic['BIN_MVISOBSC'] > 0), 'BIN_MVISOBSC'] = 1

#Confirm changes made
traffic.loc[(traffic['BIN_MVISOBSC'] > 1), 'BIN_MVISOBSC']

Series([], Name: BIN_MVISOBSC, dtype: int64)

The final column created was SERIOUS_FATALS.  This column was based on the FATALS column where FATALS (number of fatalities in a given accident) was greater than 1.  This variable is the one that I intend to use as the variable to be predicted when applying machine learning.  

In [37]:
#Create SERIOUS_FATALS column (binary yes/no greater than 1 fatality in accident) to be used as variable to be predicted
traffic['SERIOUS_FATALS'] = traffic.loc[:, 'FATALS']

#Set fatality of 1 to 0 (accident a less serious fatality)
traffic.loc[(traffic['SERIOUS_FATALS'] == 1), 'SERIOUS_FATALS'] = 0

#Confirm changes made
traffic.loc[(traffic['SERIOUS_FATALS'] == 1), 'SERIOUS_FATALS']



Series([], Name: SERIOUS_FATALS, dtype: int64)

In [38]:
#Convert remaining numbers greater than 0 to 1 (Yes)
traffic.loc[(traffic['SERIOUS_FATALS'] > 0), 'SERIOUS_FATALS'] = 1

#Confirm changes made
traffic.loc[(traffic['SERIOUS_FATALS'] > 1), 'SERIOUS_FATALS']



Series([], Name: SERIOUS_FATALS, dtype: int64)

In [39]:
#View all columns in final dataframe
traffic.columns

Index(['ST_CASE', 'MONTH', 'YEAR', 'DAY_WEEK', 'HOUR', 'RUR_URB', 'ROUTE',
       'RELJCT1', 'RELJCT2', 'WRK_ZONE', 'LGT_COND', 'WEATHER1', 'WEATHER2',
       'WEATHER', 'CF1', 'CF2', 'CF3', 'FATALS', 'DRUNK_DR', 'VEH_NO',
       'HAZ_INV', 'TRAV_SP', 'SPEEDREL', 'VSPD_LIM', 'VPROFILE', 'VPAVETYP',
       'VSURCOND', 'VTRAFCON', 'VTCONT_F', 'DEATHS', 'PER_NO', 'AGE',
       'PER_TYP', 'INJ_SEV', 'REST_USE', 'REST_MIS', 'DRUGS', 'P_SF1',
       'MDRDSTRD', 'DRIMPAIR', 'MFACTOR', 'MDRMANAV', 'MVIOLATN', 'MVISOBSC',
       'BIN_WRK_ZONE', 'BIN_SPEEDREL', 'BIN_REST_USE', 'BIN_DRUGS',
       'BIN_P_SF1', 'BIN_MDRDSTRD', 'BIN_DRIMPAIR', 'BIN_MFACTOR',
       'BIN_MDRMANAV', 'BIN_MVISOBSC', 'SERIOUS_FATALS'],
      dtype='object')

In [42]:
traffic.info()
traffic.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79619 entries, 0 to 79618
Data columns (total 55 columns):
ST_CASE           79619 non-null int64
MONTH             79619 non-null int64
YEAR              79619 non-null int64
DAY_WEEK          79619 non-null int64
HOUR              79619 non-null int64
RUR_URB           79619 non-null int64
ROUTE             79619 non-null int64
RELJCT1           79619 non-null int64
RELJCT2           79619 non-null int64
WRK_ZONE          79619 non-null int64
LGT_COND          79619 non-null int64
WEATHER1          79619 non-null int64
WEATHER2          79619 non-null int64
WEATHER           79619 non-null int64
CF1               79619 non-null int64
CF2               79619 non-null int64
CF3               79619 non-null int64
FATALS            79619 non-null int64
DRUNK_DR          79619 non-null int64
VEH_NO            79619 non-null int64
HAZ_INV           79619 non-null int64
TRAV_SP           79619 non-null int64
SPEEDREL          79619 non-null 

,ST_CASE,MONTH,YEAR,DAY_WEEK,HOUR,RUR_URB,ROUTE,RELJCT1,RELJCT2,WRK_ZONE,...,BIN_SPEEDREL,BIN_REST_USE,BIN_DRUGS,BIN_P_SF1,BIN_MDRDSTRD,BIN_DRIMPAIR,BIN_MFACTOR,BIN_MDRMANAV,BIN_MVISOBSC,SERIOUS_FATALS
0,10001,1,2015,5,2,1,3,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,10002,1,2015,5,22,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
2,10003,1,2015,5,1,1,2,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,10003,1,2015,5,1,1,2,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,10004,1,2015,1,0,1,3,0,1,0,...,0,0,0,0,0,1,0,0,0,0
